In [1]:
import os
from dotenv import load_dotenv

from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage
from langchain.agents import create_tool_calling_agent
from langchain import hub
from langchain_unstructured import UnstructuredLoader
import faiss
from langchain_community.vectorstores import FAISS
from langchain_core.tools import tool
from tqdm.autonotebook import tqdm as notebook_tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.document_loaders import DirectoryLoader


/tmp/ipykernel_31942/1986130783.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
llm = ChatOllama(model='qwen2:latest', temperature=0)
embed= OllamaEmbeddings(model='mxbai-embed-large:latest')

In [3]:
# Load, chunk and index the contents of the blog.
docs = DirectoryLoader('source/', use_multithreading=True).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [ ]:
# # index = faiss.IndexFlatL2(len(embed.embed_query("hello world")))

vector_store = FAISS.from_documents(
    documents=splits,
    embedding=embed,
)
vector_store.save_local("faiss_index")
db = FAISS.load_local("faiss_index", embed, allow_dangerous_deserialization=True)

In [5]:
# from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
# embed_2 = FastEmbedEmbeddings()

# vector_store = FAISS.from_documents(
#     documents=splits,
#     embedding=embed_2,
# )

# vector_store.save_local("faiss_index")
# db = FAISS.load_local("faiss_index", embed, allow_dangerous_deserialization=True)

2025-04-10 09:57:55.664 | ERROR    | fastembed.common.model_management:download_model:429 - Could not download model from HuggingFace: 401 Client Error. (Request ID: Root=1-67f78813-4b8e6cb81786acc0392fc93a;2b5b7f6b-f72c-495e-b8cb-86103ca6c63b)

Repository Not Found for url: https://huggingface.co/api/models/qdrant/bge-small-en-v1.5-onnx-q.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.
Invalid credentials in Authorization header Falling back to other sources.
2025-04-10 09:57:55.665 | ERROR    | fastembed.common.model_management:download_model:450 - Could not download model from either source, sleeping for 3.0 seconds, 2 retries left.
2025-04-10 09:57:58.940 | ERROR    | fastembed.common.model_management:download_model:429 - Could not download model from HuggingFace: 401 Client Error. (Request ID: Root=1-67f78816-0930b6826aa2f0437e77d6a8;fa5db851-e334-4cb1-9dc6-798652496faf)

R

ValidationError: 1 validation error for FastEmbedEmbeddings
  Value error, Could not load model BAAI/bge-small-en-v1.5 from any source. [type=value_error, input_value={'model_name': 'BAAI/bge-...s': None, 'model': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [ ]:
prompt = hub.pull('hwchase17/openai-functions-agent')

@tool(response_format="content_and_artifact")
def retrieve(query:str):
    """
    Retrieve information related to a query

    Args:
        query (str): user question or comment
    """
    retrieved_docs = vector_store.similarity_search(query, k=1)
    serialized = "".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

@tool(response_format="content")
def ip_address():
    """
    Retrieve IP address information of the user
    """
    from requests import get

    ip = get('https://api.ipify.org').text
    return ip

tools = [retrieve, ip_address]
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({'input':'What is the full concept of MLOps?'})
print(response['output'])

In [ ]:
response = agent_executor.invoke({'input':'what is my ip address?'})
print(response['output'])

In [ ]:
class Agentic():
    